In [3]:
import os, sys
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import imageio
import pprint

import matplotlib.pyplot as plt
import torch
import mcubes
import trimesh
import run_nerf
import run_nerf_helpers



In [4]:
basedir = './logs_NEW_NORM'
expname = 'kitti360_test_binocular300000'

config = os.path.join(basedir, expname, 'config.txt')
print('Args:')
print(open(config, 'r').read())

Args:
expname = kitti360_test_binocular
basedir = ./logs_NEW_NORM
datadir = /media/ry/483BED215A2D2EBA/KITTI-360
dataset_type = KITTI360

no_batching = False
use_viewdirs = True
white_bkgd = False
lrate_decay = 500
render_test = True

N_samples = 64
N_importance = 128
N_rand = 1024

precrop_iters = 500
precrop_frac = 0.5

half_res = False



In [7]:
parser = run_nerf.config_parser()
ft_str = '' 
ft_str = '--ft_path {}'.format(os.path.join(basedir, expname, 'model_300000.npy'))
args = parser.parse_args('--config {} '.format(config) + ft_str)


In [ ]:
# Create nerf model
_, render_kwargs_test, start, grad_vars, models = run_nerf.create_nerf(args)


near = 2.
far = 6.
bds_dict = {
        'near' : near,
        'far' : far,
    }
render_kwargs_test.update(bds_dict)

print('Render kwargs:')
pprint.pprint(render_kwargs_test)

net_fn = render_kwargs_test['network_query_fn']
print(net_fn)

# Render an overhead view to check model was loaded correctly
c2w = np.eye(4)[:3,:4].astype(np.float32) # identity pose matrix
c2w[2,-1] = 4.
H, W, focal = 800, 800, 1200.
down = 8
test = run_nerf.render(H//down, W//down, focal/down, c2w=c2w, **render_kwargs_test)
img = np.clip(test[0],0,1)
plt.imshow(img)
plt.show()